# Vroom Data Engineering / API Pull

In [4]:
import sqlite3
from pandas.io import sql

conn = sqlite3.connect('/Users/adamowens/datascience/vault-econ/vroom/vroom.db')
c = conn.cursor()

In [39]:
import pandas as pd
import unicodedata as uni
df = pd.read_sql_query('select * from pricecomp;', conn)

#for row in c.execute('SELECT make FROM pricecomp;'):
#    print row


In [40]:
make = []
for row in list(df['make']):
    make.append(str(row))

year = []
for row in list(df['year']):
    year.append(str(row))
    
model = []
for row in list(df['model']):
    model.append(str(row))
    
mileage = []
for row in list(df['miles']):
    mileage.append(str(row))

In [41]:
# Create Dictionary
model_dict = {'GLA':'GLA-Class', 'SLK':'SLK-Class', 'CLA':'CLA-Class', 'G37 Sedan': 'GSedan', \
              'MKT Town Car': 'MKT', 'Ram Pickup 2500': '2500', 'MKZ Hybrid':'MKZ','MAZDA3':'3',\
              'Ram Pickup 1500':'1500','Ram Pickup 3500': '3500','Expedition EL':'Expedition',\
              'Promaster Cargo':'PromasterCargoVan','Corvette':'CorvetteStingray',\
              'Wrangler Unlimited':'Wrangler', 'Express Passenger':'Express','ProMaster Cargo':'PromasterCargoVan'\
             }

In [43]:
import urllib
import urllib2
import json

edmunds_api_key = 'kdj6mj65bapsf8wzc936j36u'

def edmunds_styleID(make, model, year):
    url = 'https://api.edmunds.com/api/vehicle/v2/' + make + '/' + model + '/' + year + '/'+ \
    '?fmt=json&api_key=' + edmunds_api_key
    json_obj = urllib2.urlopen(url)
    data = json.load(json_obj)
    for item in data['styles']:
        return item['id']

In [44]:
condition = 'Clean' 
zip_code = '75052'

def edmunds_TMV(styleID, mileage):
    url = 'https://api.edmunds.com/v1/api/tmv/tmvservice/calculateusedtmv?styleid='\
    +styleID+'&condition='+ condition +'&mileage='+mileage+'&zip='+zip_code+'&fmt=json&api_key='+edmunds_api_key
    json_obj = urllib2.urlopen(url)
    data = json.load(json_obj)
    return data.get('tmv').get('nationalBasePrice').get('usedTmvRetail')


In [10]:
from urllib2 import HTTPError
import re

styleID = []

for x, y, z in zip(make, model, year):
    try: styleID.append(str(edmunds_styleID(x,y,z)))
    except HTTPError:
        y2 = re.sub(r"\s+", "", y)
        try: styleID.append(str(edmunds_styleID(x,y2,z)))
        except HTTPError:
            y_replace = str(model_dict.get(y))
            try: styleID.append(str(edmunds_styleID(x,y_replace,z)))
            except HTTPError:
                styleID.append("NA")


In [24]:
usedTmvRetail = []
for x, y in zip(styleID, mileage):
    try: usedTmvRetail.append(str(edmunds_TMV(x,y)))
    except HTTPError:
        usedTmvRetail.append("None")

In [38]:
import pandas as pd

usedTMVRetail = ['' if x is None else x for x in usedTmvRetail]
df_results = pd.DataFrame({'styleID':styleID,'usedTMVRetail':usedTMVRetail})
df_results = df_results.replace(['None'], '') # Replace the 'None' values with blanks for easier computations

df_total = pd.concat([df, df_results], axis = 1)

df_total.to_csv('vroom_pricecomp')


In [13]:
# write to database
from pandas.io import sql

#df_total.to_sql(name='vroom_total', flavor = 'sqlite',con=conn)